In [144]:
import os
import numpy as np
from scipy import misc
import argparse
import json
import cv2
import shutil
from tensorpack import *
from train import Model
from reader import Data
from cfgs.config import cfg


from matplotlib import pyplot as plt
%matplotlib inline

In [153]:
DATA_ROOT = "doodles"
animal_name = "koala"
# pred_ori_result = "animal_pred_ori_resul_train"
pred_ori_result = "animal_pred_ori_result_test"


In [154]:
# initialize the model
depth = 18
model = Model(depth)
sess_init = SaverRestore("train_log/train0728-111507/model-1")
predict_config = PredictConfig(session_init=sess_init,
                               model=model,
                               input_names=["input"],
                               output_names=["logits"])

In [147]:
predict_func = OfflinePredictor(predict_config)

[0728 17:13:42 @regularize.py:18] Apply regularizer for conv0/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group0/block0/conv1/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group0/block0/conv2/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group0/block1/conv1/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group0/block1/conv2/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group1/block0/conv1/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group1/block0/conv2/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group1/block0/convshortcut/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group1/block1/conv1/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group1/block1/conv2/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group2/block0/conv1/W:0
[0728 17:13:42 @regularize.py:18] Apply regularizer for group2/block0/conv2/W:0
[0728 17:13:42 @regularize.py:18] Apply regular

In [155]:
def get_data(filename):
    with open(filename,'r') as f:
        lin = f.readlines()
    rc = [ele.split(' ') for ele in lin]
    return rc

In [156]:
 
if pred_ori_result.endswith("train") == True:
    pred_ori_result = "animal_pred_ori_resul_train"
    flist = cfg.train_list 
else:
    pred_ori_result = "animal_pred_ori_result_test"
    flist = cfg.test_list

In [157]:
name_list = [flist] if type(flist) is not list else flist

In [158]:
files =[]
for list_ in name_list:
    files.extend(get_data(list_))

In [160]:
file_path = os.path.join(DATA_ROOT,animal_name)
    
conf_path = os.path.join(file_path,"conf.json")

resutl_path = os.path.join(DATA_ROOT,pred_ori_result) 



if os.path.isdir(resutl_path) == True:
    shutil.rmtree(resutl_path)
os.mkdir(resutl_path)
# print(files[0])

for file in files:
#     if "mask" in file or "json" in file or "result" in file:
#         continue
    img_path = file[0]
    img = misc.imread(img_path)
    # plt.imshow(img)
    input_img = np.expand_dims(img, axis=0)
    predictions = predict_func([input_img])[0][0]
    norm_bones = predictions + np.asarray(cfg.anchor_bones)
    
    
    bboxes = []
    for label_idx in range(cfg.obj_num):
        mask_path = img_path.replace("png", "mask." + str(label_idx) + ".png")
        cur_label_img = misc.imread(mask_path, mode='L')
        cur_label_img = np.minimum(cur_label_img, np.ones(cur_label_img.shape))

        t = np.nonzero(cur_label_img)
        ymin = np.min(t[0])
        ymax = np.max(t[0])
        xmin = np.min(t[1])
        xmax = np.max(t[1])
        bboxes.append([xmin, ymin, xmax, ymax])
    bones = []
    i = 0
    for box_idx in cfg.match:
        norm_bone = norm_bones[i:i+2]
        i += 2
        [xmin, ymin, xmax, ymax] = bboxes[box_idx]
        xcenter = (xmax + xmin) / 2
        ycenter = (ymax + ymin) / 2
        box_width = (xmax - xmin)
        box_height = (ymax - ymin)
        x = norm_bone[0] * box_width + xcenter
        y = norm_bone[1] * box_height + ycenter
        bones.append([int(x), int(y)])
    
   
    for bone in bones:
        img = cv2.circle(img, (bone[0], bone[1]), 2, (0, 255, 255), thickness=2, lineType=8, shift=0)
    
    json_file = img_path.replace(".png",".json")
#     print(img_path+json_file)
    if os.path.exists(json_file) == False:
        continue
    with open((json_file),"r") as f:
        lines = json.load(f)
     
    for line in lines:
        img = cv2.circle(img, (line[0], line[1]), 2, (255, 0, 0), thickness=2, lineType=8, shift=0)
#     plt.imshow(img)
    result = os.path.join(resutl_path,img_path.split("/")[-1])
#     print(result)?
    if os.path.isfile(result) == True:
        os.remove(result)
    cv2.imwrite(result,img)
#     break
    